In [12]:
 from google.colab import drive
 import os

# README - Execute this cell to mount the notebook in your google drive. 
# Execute the cell and follow the link to sign and, paste the given key in the little text box. The credentials are only available for you. 

drive.mount('/content/drive', force_remount=True)

if not os.path.exists("/content/drive/MyDrive/audio-dataset"):
  print("Pulling dataset")
  os.mkdir("/content/drive/MyDrive/audio-dataset")
  !git clone https://github.com/emilstahl97/Audio-dataset.git
else:
  print("Dataset already exists")

#os.chdir("/content/drive/MyDrive/audio-dataset/Audio-dataset/Audio-dataset")
os.chdir("/content/drive/MyDrive/audio-dataset/Audio-dataset/Rawdata")

!git pull
!ls

RAVDESS_PATH = "./RAVDESS"
SAVEE_PATH = "./SAVEE"
SAVED_MODELS_PATH = "../saved_models"


Mounted at /content/drive
Dataset already exists
Your configuration specifies to merge with the ref 'refs/heads/main'
from the remote, but no such ref was fetched.
03-01-01-01-01-01-01.wav  03-01-04-01-01-01-01.wav  03-01-06-02-01-01-01.wav
03-01-01-01-01-01-02.wav  03-01-04-01-01-01-02.wav  03-01-06-02-01-01-02.wav
03-01-01-01-01-01-03.wav  03-01-04-01-01-01-03.wav  03-01-06-02-01-01-03.wav
03-01-01-01-01-01-04.wav  03-01-04-01-01-01-04.wav  03-01-06-02-01-01-04.wav
03-01-01-01-01-01-06.wav  03-01-04-01-01-01-06.wav  03-01-06-02-01-01-06.wav
03-01-01-01-01-01-07.wav  03-01-04-01-01-01-07.wav  03-01-06-02-01-01-07.wav
03-01-01-01-01-01-08.wav  03-01-04-01-01-01-08.wav  03-01-06-02-01-01-08.wav
03-01-01-01-01-01-09.wav  03-01-04-01-01-01-09.wav  03-01-06-02-01-01-09.wav
03-01-01-01-01-01-10.wav  03-01-04-01-01-01-10.wav  03-01-06-02-01-01-10.wav
03-01-01-01-01-01-11.wav  03-01-04-01-01-01-11.wav  03-01-06-02-01-01-11.wav
03-01-01-01-01-01-12.wav  03-01-04-01-01-01-12.wav  03-01-06-02-01

##**PIP**

In [13]:
!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install --upgrade librosa
!pip install --upgrade scipy
!pip install --upgrade scikit-learn
!pip install --upgrade pandas

##**IMPORTING**

In [14]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
import pandas as pd
import sys

print("Everything imported")

Everything imported


## **Make list**

In [15]:
mylist= os.listdir('./')
type(mylist)
print("Number of audio files: {}".format(len(mylist)))

print(mylist[1][6:-16])



Number of audio files: 1380
03


##**Get labels**

In [16]:
feeling_list=[]
for item in mylist:
    if item[6:-16]=='02' and int(item[18:-4])%2==0:
        feeling_list.append('female_calm')
    elif item[6:-16]=='02' and int(item[18:-4])%2==1:
        feeling_list.append('male_calm')
    elif item[6:-16]=='03' and int(item[18:-4])%2==0:
        feeling_list.append('female_happy')
    elif item[6:-16]=='03' and int(item[18:-4])%2==1:
        feeling_list.append('male_happy')
    elif item[6:-16]=='04' and int(item[18:-4])%2==0:
        feeling_list.append('female_sad')
    elif item[6:-16]=='04' and int(item[18:-4])%2==1:
        feeling_list.append('male_sad')
    elif item[6:-16]=='05' and int(item[18:-4])%2==0:
        feeling_list.append('female_angry')
    elif item[6:-16]=='05' and int(item[18:-4])%2==1:
        feeling_list.append('male_angry')
    elif item[6:-16]=='06' and int(item[18:-4])%2==0:
        feeling_list.append('female_fearful')
    elif item[6:-16]=='06' and int(item[18:-4])%2==1:
        feeling_list.append('male_fearful')
    elif item[:1]=='a':
        feeling_list.append('male_angry')
    elif item[:1]=='f':
        feeling_list.append('male_fearful')
    elif item[:1]=='h':
        feeling_list.append('male_happy')
    #elif item[:1]=='n':
        #feeling_list.append('neutral')
    elif item[:2]=='sa':
        feeling_list.append('male_sad')


#print(feeling_list)
labels = pd.DataFrame(feeling_list)
#labels[:10]

## Getting the features of audio files using librosa

In [17]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
        X, sample_rate = librosa.load('./'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1



#df[:5]
#df3 = pd.DataFrame(df['feature'].values.tolist())
#df3[:5]

In [18]:
df3 = pd.DataFrame(df['feature'].values.tolist())

newdf = pd.concat([df3,labels], axis=1)

rnewdf = newdf.rename(index=str, columns={"0": "label"})

print(rnewdf)

from sklearn.utils import shuffle
rnewdf = shuffle(newdf)

rnewdf=rnewdf.fillna(0)

           0          1          2    ...        214        215           0  
0   -53.306465 -51.239155 -50.094662  ... -56.696125 -58.832737    male_happy
1   -52.999336 -52.908321 -52.419476  ... -53.492561 -53.001705  female_happy
2   -53.141987 -52.674992 -50.214226  ... -56.517986 -57.023308    male_happy
3   -64.933876 -64.933876 -64.933876  ... -64.933876 -64.933876  female_happy
4   -60.311516 -59.178108 -59.345013  ... -47.422729 -48.096535    male_happy
..         ...        ...        ...  ...        ...        ...           ...
915 -48.113033 -48.335693 -50.188248  ... -47.045372 -48.921272  female_happy
916 -60.501297 -62.577785 -60.446762  ... -52.179653 -53.992882    male_happy
917 -59.302078 -57.921028 -56.350586  ... -54.675850 -56.525337  female_happy
918 -57.354610 -56.764095 -56.161160  ... -59.327217 -62.622055    male_happy
919 -57.820358 -57.820358 -57.820358  ... -57.820358 -57.820358  female_happy

[920 rows x 217 columns]


## Dividing the data into test and train

In [19]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

print(train)

           0          1          2    ...        214        215             0  
246 -55.102558 -56.048286 -58.384499  ... -51.789467 -52.396381        male_sad
219 -61.465416 -61.465416 -61.465416  ... -57.031345 -60.724686      female_sad
298 -54.135109 -54.135109 -54.135109  ... -53.462734 -53.641254    female_angry
579 -50.567917 -51.057613 -50.570278  ... -53.014458 -53.183506  female_fearful
786 -63.396774 -61.330418 -59.724636  ... -65.806625 -65.705292       male_calm
..         ...        ...        ...  ...        ...        ...             ...
13  -53.769718 -53.081631 -53.512714  ... -55.381798 -58.272831    female_happy
648 -70.267769 -70.267769 -70.267769  ... -61.082741 -60.234661       male_calm
214 -44.627983 -46.197441 -44.684441  ... -39.757011 -41.383408        male_sad
907 -64.447998 -64.447998 -64.447998  ... -52.965984 -52.242493    female_happy
93  -47.371452 -47.371452 -47.371452  ... -47.371452 -47.371452    female_happy

[744 rows x 217 columns]


In [20]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [21]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
y_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
X_train.shape

(744, 216)

# Padding sequence for CNN model

In [24]:
print('Pad sequences')
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

Pad sequences


In [25]:
import tensorflow

model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = tensorflow.keras.optimizers.RMSprop(learning_rate =0.00001, decay=1e-6)

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 216, 128)          768       
                                                                 
 activation (Activation)     (None, 216, 128)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 216, 128)          82048     
                                                                 
 activation_1 (Activation)   (None, 216, 128)          0         
                                                                 
 dropout (Dropout)           (None, 216, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 27, 128)          0         
 )                                                               
                                                        

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [28]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=32, epochs=1009, validation_data=(x_testcnn, y_test))

Epoch 1/1009
24/24 [==============================] - 14s 41ms/step - loss: 2.6771 - accuracy: 0.1102 - val_loss: 2.3284 - val_accuracy: 0.1420
Epoch 2/1009
24/24 [==============================] - 0s 17ms/step - loss: 2.4680 - accuracy: 0.1142 - val_loss: 2.2631 - val_accuracy: 0.1307
Epoch 3/1009
24/24 [==============================] - 0s 16ms/step - loss: 2.3667 - accuracy: 0.1237 - val_loss: 2.2702 - val_accuracy: 0.1364
Epoch 4/1009
24/24 [==============================] - 0s 16ms/step - loss: 2.3599 - accuracy: 0.1048 - val_loss: 2.2552 - val_accuracy: 0.1307
Epoch 5/1009
24/24 [==============================] - 0s 17ms/step - loss: 2.3364 - accuracy: 0.1358 - val_loss: 2.2285 - val_accuracy: 0.1477
Epoch 6/1009
24/24 [==============================] - 0s 16ms/step - loss: 2.3044 - accuracy: 0.1465 - val_loss: 2.2158 - val_accuracy: 0.1250
Epoch 7/1009
24/24 [==============================] - 0s 15ms/step - loss: 2.2493 - accuracy: 0.1653 - val_loss: 2.2195 - val_accuracy: 0.130

## Plotting the accuracy and loss graph

In [36]:
#sigmoid
print(cnnhistory.history)
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

{'loss': [2.677144765853882, 2.4680373668670654, 2.3667383193969727, 2.359930992126465, 2.3364460468292236, 2.3044209480285645, 2.2493226528167725, 2.264207601547241, 2.2395358085632324, 2.2414557933807373, 2.2244138717651367, 2.2034831047058105, 2.183342456817627, 2.177473783493042, 2.1858274936676025, 2.1515166759490967, 2.1560397148132324, 2.1742324829101562, 2.1571836471557617, 2.1310486793518066, 2.100846290588379, 2.083409547805786, 2.0913236141204834, 2.0896828174591064, 2.0618977546691895, 2.050916910171509, 2.0462963581085205, 2.0475387573242188, 2.052065372467041, 2.014385461807251, 2.00087571144104, 2.014049530029297, 1.9901784658432007, 1.9992824792861938, 1.9765567779541016, 1.9881893396377563, 1.950476884841919, 1.9642847776412964, 1.9409233331680298, 1.9376487731933594, 1.9386932849884033, 1.9207054376602173, 1.896677017211914, 1.9222556352615356, 1.898451566696167, 1.873759388923645, 1.861983060836792, 1.8618996143341064, 1.8623074293136597, 1.8515937328338623, 1.829684

ImportError: ignored

<Figure size 432x288 with 1 Axes>

## Save model

In [31]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), "../saved_models")
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/drive/My Drive/audio-dataset/Audio-dataset/Rawdata/../saved_models/Emotion_Voice_Detection_Model.h5 
